<a href="https://colab.research.google.com/github/SecretServicess/Disease-Prediction-and-QA-system-in-Healthcare-Domain/blob/main/QuestionAnsweringSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import warnings
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
warnings.filterwarnings('ignore')

C:\Users\Prakhar Jadaun\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
file1 = pd.read_json('file1.json')
file1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   answer    171 non-null    object
 1   question  171 non-null    object
 2   url       171 non-null    object
 3   tags      171 non-null    object
dtypes: object(4)
memory usage: 5.5+ KB


In [ ]:
file2 = pd.read_json('file2.json')
file2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137052 entries, 0 to 137051
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   question      137052 non-null  object
 1   short_answer  137052 non-null  object
 2   answer        137052 non-null  object
 3   tags          137052 non-null  object
 4   url           137052 non-null  object
dtypes: object(5)
memory usage: 5.2+ MB


In [ ]:
file3 = pd.read_json('file3.json')
file3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   answer         465 non-null    object
 1   question       465 non-null    object
 2   question_text  465 non-null    object
 3   tags           465 non-null    object
 4   url            465 non-null    object
dtypes: object(5)
memory usage: 18.3+ KB


In [ ]:
file4 = pd.read_json('file4.json')
file4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tags      22 non-null     object
 1   question  22 non-null     object
 2   answer    22 non-null     object
 3   context   22 non-null     object
dtypes: object(4)
memory usage: 832.0+ bytes


In [ ]:
file5 = pd.read_json('file5.json')
file5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5679 entries, 0 to 5678
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   answer         5679 non-null   object
 1   answer_author  5679 non-null   object
 2   question       5679 non-null   object
 3   question_text  5679 non-null   object
 4   tags           5679 non-null   object
 5   url            5679 non-null   object
dtypes: object(6)
memory usage: 266.3+ KB


In [ ]:
file6 = pd.read_json('file6.json')
file6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23437 entries, 0 to 23436
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   answer    23437 non-null  object
 1   question  23437 non-null  object
 2   url       23437 non-null  object
 3   tags      23437 non-null  object
dtypes: object(4)
memory usage: 732.5+ KB


In [ ]:
# Load each JSON file into a pandas dataframe
file1_df = pd.read_json('file1.json')
file2_df = pd.read_json('file2.json')
file3_df = pd.read_json('file3.json')
file4_df = pd.read_json('file4.json')
file5_df = pd.read_json('file5.json')
file6_df = pd.read_json('file6.json')

# Merge the dataframes on the common column names
merged_df = pd.concat([file1_df, file2_df, file3_df, file4_df, file5_df, file6_df], axis=0, ignore_index=True)

# Write the merged dataframe to a new JSON file
# merged_df.to_json('merged.json', orient='records')

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166826 entries, 0 to 166825
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   answer         166826 non-null  object
 1   question       166826 non-null  object
 2   url            166804 non-null  object
 3   tags           166826 non-null  object
 4   short_answer   137052 non-null  object
 5   question_text  6144 non-null    object
 6   context        22 non-null      object
 7   answer_author  5679 non-null    object
dtypes: object(8)
memory usage: 10.2+ MB


In [ ]:
merged_df.isna().sum()

answer                0
question              0
url                  22
tags                  0
short_answer      29774
question_text    160682
context          166804
answer_author    161147
dtype: int64

In [ ]:
merged_df.tail()

,answer,question,url,tags,short_answer,question_text,context,answer_author
166821,to avoid unintentional overdoses among adults ...,how can accidental of acetaminophen overdose b...,http://answers.webmd.com/answers/1175997/how-c...,"[drug overdose, acetaminophen]",NaN,NaN,NaN,NaN
166822,if you take more medication than you have been...,what should i do if i take an overdose of maxalt?,http://answers.webmd.com/answers/1168806/what-...,[drug overdose],NaN,NaN,NaN,NaN
166823,call your doctor or poison control center or g...,what do i do in case of an overdose of relpax?,http://answers.webmd.com/answers/1168730/what-...,[drug overdose],NaN,NaN,NaN,NaN
166824,in the u. s. suicide attempts account for over...,is overdose with acetaminophen usually acciden...,http://answers.webmd.com/answers/1175998/is-ov...,"[drug overdose, acetaminophen]",NaN,NaN,NaN,NaN
166825,the answer is that liver damage from acetamino...,how does an overdose of acetaminophen cause li...,http://answers.webmd.com/answers/1176004/how-d...,"[drug overdose, injury, liver, acetaminophen]",NaN,NaN,NaN,NaN


In [ ]:
merged_df.columns

Index(['answer', 'question', 'url', 'tags', 'short_answer', 'question_text',
       'context', 'answer_author'],
      dtype='object')

In [ ]:
col = ['answer','question']
merged_df = merged_df[col]
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166826 entries, 0 to 166825
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   answer    166826 non-null  object
 1   question  166826 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


**Preprocessing**

In [ ]:
# performing basic preprocessing on data
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # tokenizing the words
    tokens = word_tokenize(text.lower())
    # removing the stop words from the data and performing lemmatization on each token
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    # returning the filtered tokens
    return filtered_tokens
# creating a column containing processed question column
merged_df['processed_question'] = merged_df['question'].apply(preprocess_text)

In [ ]:
# defining a function to compute similarity between the user input and the processed question
def compute_similarity(user_input, processed_question):
    # creating an instance of tfidf vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    # fitting the user input on the tfidf vectorizer
    tfidf_user_input = tfidf_vectorizer.fit_transform([user_input])
    # transforming the column processed question
    tfidf_processed_question = tfidf_vectorizer.transform([' '.join(processed_question)])
    # returning the cosine similarity between the user input and the question column
    return cosine_similarity(tfidf_user_input, tfidf_processed_question)[0][0]

In [ ]:
# getting the input from the user
print("Question?".center(70,'-'))
user_input = input()

# preprocessing the user's input
processed_user_input = preprocess_text(user_input)

# compute similarity between the user's input and each question in the dataset
merged_df['similarity'] = merged_df['processed_question'].apply(lambda x: compute_similarity(' '.join(processed_user_input), x))

# return the answer to the most similar question
most_similar_index = merged_df['similarity'].idxmax()
print("ANSWER".center(70,'-'))
print(merged_df.loc[most_similar_index]['answer'])

------------------------------Question?-------------------------------
--------------------------------ANSWER--------------------------------
sure it is (was) a very common medication. if you are in il call me. most of the times however i use wellbutrin (bupropion) - less side effects additional benefits and the same likelihood to quit. by the way it is better to be used as a part of a "cocktail" and with a couch-like person.
